In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum as sum_, first, round, min, when, lit, to_date, to_timestamp, date_format
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, TimestampType
from pyspark.sql.functions import col, year, month, when, avg, round
from pyspark.sql.functions import col, year, month, when, avg, round, min


# Configurações do Spark
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

# Criar sessão Spark
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Lista de arquivos CSV a serem lidos
csv_files = [
    'dados_2025_06_02.csv',
    'dados_2025_05_27.csv',
    'dados_2025_05_28.csv',
    'dados_2025_05_29.csv',
    'dados_2025_05_30.csv',
    'dados_2025_05_31.csv',
    'dados_2025_06_01.csv',
    'dados_2025_05_20.csv',
    'dados_2025_05_21.csv',
    'dados_2025_05_22.csv',
    'dados_2025_05_23.csv',
    'dados_2025_05_24.csv',
    'dados_2025_05_25.csv',
    'dados_2025_05_26.csv'
]

# Prefixo do caminho S3
s3_prefix = 's3a://bucket-raw-upa-connect-gabriel/'

# Criar a lista completa de caminhos S3
s3_paths = [s3_prefix + file for file in csv_files]

# Ler os CSVs necessários a partir da lista de caminhos
TabelaCompleta = spark.read.option('delimiter', ',') \
                             .option('header', 'true') \
                             .option('nullValue', 'null') \
                             .csv(s3_paths) # Agora lê múltiplos arquivos

# Ler o CSV, já tratando "NULL" como valor nulo
upa_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-gabriel/upa.csv')

paciente_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-gabriel/paciente.csv')

sensor_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-gabriel/sensor.csv')


unidadeDeMedida_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-gabriel/UnidadeDeMedida.csv')

temperaturaAmbiente_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-raw-upa-connect-gabriel/clima_tempo.csv')

# Converter 'valor' para DoubleType e 'fk_upa' para IntegerType
# a = a.withColumn('valor', col('valor').cast(DoubleType())) \
#      .withColumn('fk_paciente', col('fk_paciente').cast(IntegerType()))

# TabelaCompleta.show()
# upa_df.show()
# paciente_df.show(170)
# sensor_df.show()
# unidadeDeMedida_df.show()




In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, avg, round, to_date, to_timestamp
from pyspark.sql.types import DoubleType, TimestampType

# Configurações do Spark
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

# Criar sessão Spark
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Ler CSVs
#TabelaCompleta = spark.read.option('delimiter', ',').option('header', 'true').option('nullValue', 'null') \
#    .csv('s3a://bucket-raw-upa-connect-gabriel/tabela_historico_sensor.csv')

temperaturaAmbiente_df = spark.read.option('delimiter', ',').option('header', 'true') \
    .csv('s3a://bucket-raw-upa-connect-gabriel/clima_tempo.csv')

# =============================
# 1. Processar temperatura dos pacientes
# =============================
df_filtrado_inicial = TabelaCompleta.filter((col("fk_sensor") == 4)) \
    .filter(col('valor').isNotNull())

condicao_sensor_temp = (col('fk_sensor') == 4) & (col('valor') >= 34) & (col('valor') <= 42)

df_filtrado_final = df_filtrado_inicial.filter(condicao_sensor_temp) \
    .withColumn('valor', col('valor').cast(DoubleType())) \
    .withColumn('data_hora', col('data_hora').cast(TimestampType()))

df_resultado_final = df_filtrado_final.withColumn('ano', year(col('data_hora'))) \
    .withColumn('mes', month(col('data_hora'))) \
    .groupBy('ano', 'mes') \
    .agg(round(avg(col('valor')), 1).alias('Temperatura_Media_Paciente')) \
    .orderBy('ano', 'mes')

# =============================
# 2. Processar temperatura ambiente mensal
# =============================
temperaturaAmbiente_df = temperaturaAmbiente_df \
    .withColumn('data', to_date(col('data'), 'yyyy-MM-dd')) \
    .withColumn('temperatura_media', col('temperatura_media').cast(DoubleType())) \
    .withColumn('ano', year(col('data'))) \
    .withColumn('mes', month(col('data')))

temperaturaAmbiente_df_filtrada = temperaturaAmbiente_df.filter(col('temperatura_media').isNotNull())

df_ambiente_mensal = temperaturaAmbiente_df_filtrada.groupBy('ano', 'mes') \
    .agg(round(avg('temperatura_media'), 1).alias('Temperatura_Media_Ambiente'))

# =============================
# 2.1 Processar quantidade de pessoas na UPA (sensor fk = 1)
# =============================
df_pessoas_upa = TabelaCompleta.filter((col("fk_sensor") == 1) & col("valor").isNotNull()) \
    .withColumn('valor', col('valor').cast(DoubleType())) \
    .withColumn('data_hora', col('data_hora').cast(TimestampType())) \
    .withColumn('ano', year(col('data_hora'))) \
    .withColumn('mes', month(col('data_hora')))

df_pessoas_mensal = df_pessoas_upa.groupBy('ano', 'mes') \
    .agg(round(avg('valor'), 2).alias('Quantidade_Media_Pessoas_UPA'))

# =============================
# 3. Juntar os três DataFrames e substituir null por 0
# =============================
df_final_completo = df_resultado_final \
    .join(df_ambiente_mensal, on=['ano', 'mes'], how='left') \
    .join(df_pessoas_mensal, on=['ano', 'mes'], how='left') \
    .fillna(0) \
    .orderBy('ano', 'mes')

# =============================
# 4. Exibir resultado final
# =============================
df_final_completo.show(100)


In [ ]:
df_final_completo.coalesce(1) \
        .write \
        .option('header', 'true') \
        .mode('overwrite') \
        .csv('s3a://bucket-client-upa-connect-gabriel/grafico_temperatura_paciente.csv')

In [ ]:
%pip install gspread google-auth-oauthlib boto3 pandas

In [ ]:
!pip uninstall gspread -y
!pip install gspread==5.11.3

In [ ]:
%pip install --upgrade pip

In [ ]:
# --- Instalação de bibliotecas (adicione esta seção no início do seu script) ---
import subprocess
import sys

# Função para instalar pacotes
def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"'{package}' instalado com sucesso.")
    except Exception as e:
        print(f"Erro ao instalar '{package}': {e}")
        # Em ambientes de produção, você pode querer adicionar um exit(1) aqui
        # para parar a execução se uma dependência crítica não puder ser instalada.

# Instalar as bibliotecas necessárias
install_package("pandas")
install_package("gspread")
install_package("google-auth-oauthlib") # Necessário para google.oauth2.service_account.Credentials
install_package("boto3") # Para interagir com o S3
# --- Fim da seção de instalação de bibliotecas ---


# --- CÓDIGO PARA ENVIAR PARA O GOOGLE SHEETS ---
import pandas as pd # <-- MOVIDO PARA AQUI PARA GARANTIR QUE 'pd' ESTEJA NO ESCOPO
import gspread
from google.oauth2.service_account import Credentials
import boto3
import os
import sys

# === CONFIGURAÇÕES PARA O GOOGLE SHEETS ===
GOOGLE_SHEET_ID = '1i6BfuZXPOcTp6BFAiVkpktOBym0HtakZacUKfDzu1zI'
S3_BUCKET_CREDENTIALS = 'bucket-client-upa-connect-gabriel'
S3_KEY_CREDENTIALS = 'credenciais.json'
LOCAL_CREDENTIALS_PATH = '/tmp/credenciais.json'

# === AUTENTICAÇÃO GOOGLE ===
print(f"Baixando credenciais do S3: s3://{S3_BUCKET_CREDENTIALS}/{S3_KEY_CREDENTIALS} para {LOCAL_CREDENTIALS_PATH}...")
s3 = boto3.client('s3')
try:
    s3.download_file(S3_BUCKET_CREDENTIALS, S3_KEY_CREDENTIALS, LOCAL_CREDENTIALS_PATH)
    print("Credenciais baixadas com sucesso.")
except Exception as e:
    print(f"Erro ao baixar credenciais do S3: {e}")
    sys.exit(1)

scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
credenciais = Credentials.from_service_account_file(LOCAL_CREDENTIALS_PATH, scopes=scopes)
cliente = gspread.authorize(credenciais)
planilha = cliente.open_by_key(GOOGLE_SHEET_ID)

# === ENVIA O DATAFRAME FINAL PARA A NOVA ABA "umidade" ===
NOME_ABA_TESTE = "temperatura_paciente"
print(f"Enviando DataFrame final para a aba '{NOME_ABA_TESTE}'...")

# Converter Spark DataFrame para Pandas DataFrame
try:
    df_pandas = df_final_completo.toPandas()
    print("DataFrame Spark convertido para Pandas com sucesso.")
except Exception as e:
    print(f"Erro ao converter Spark DataFrame para Pandas: {e}")
    print("O DataFrame pode ser muito grande para a memória do driver ou a sessão Spark está inválida.")
    sys.exit(1)

# --- NOVO TRECHO: Ajustar tipagem das colunas para o Google Sheets ---
# Apenas formata colunas de data/hora para string ISO 8601.
# As demais colunas (numéricas, strings, etc.) são mantidas em seus tipos nativos no Pandas.
# O gspread e o Google Sheets farão a inferência de tipo para elas.

# # Coluna 'data_chegada': assegurar que é string DD/MM/YYYY
if 'data' in df_pandas.columns:
    if pd.api.types.is_datetime64_any_dtype(df_pandas['data']):
        df_pandas['data'] = df_pandas['data'].dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    elif pd.api.types.is_string_dtype(df_pandas['data']):
        # Tentar converter strings que podem não estar em formato padrão para datetime,
        # e depois formatar para DD/MM/YYYY. 'errors=coerce' transforma não-datas em NaT.
        df_pandas['data'] = pd.to_datetime(df_pandas['data'], errors='coerce').dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    # Preencher valores nulos (NaT) com string vazia, pois None também pode causar problemas.
    df_pandas['data'] = df_pandas['data'].fillna('')

# # Coluna 'horario_chegada': assegurar que é string HH:MM:SS
if 'hora' in df_pandas.columns:
    if pd.api.types.is_datetime64_any_dtype(df_pandas['hora']):
        df_pandas['hora'] = df_pandas['hora'].dt.strftime('%H:%M:%S')
    elif pd.api.types.is_string_dtype(df_pandas['hora']):
        # Se já é string (como no seu caso vindo do Spark), apenas garanta que nulos sejam preenchidos.
        df_pandas['hora'] = df_pandas['hora'].fillna('')


# Nenhuma conversão global df_pandas.astype(str) é necessária aqui.
# gspread é capaz de lidar com int, float, bool, e strings diretamente.

dados = [df_pandas.columns.tolist()] + df_pandas.values.tolist()


try:
    aba = planilha.worksheet(NOME_ABA_TESTE)
    aba.clear()
    print(f"Aba '{NOME_ABA_TESTE}' encontrada e limpa.")
except gspread.exceptions.WorksheetNotFound:
    aba = planilha.add_worksheet(title=NOME_ABA_TESTE, rows=str(len(dados) + 100), cols=str(len(df_pandas.columns) + 10))
    print(f"Aba '{NOME_ABA_TESTE}' criada.")

try:
    aba.update('A1', dados)
    print(f"Dados do DataFrame final enviados com sucesso para a aba '{NOME_ABA_TESTE}'.")
except Exception as e:
    print(f"Erro ao atualizar a aba do Google Sheets: {e}")

# Opcional: Remover o arquivo de credenciais temporário
if os.path.exists(LOCAL_CREDENTIALS_PATH):
    os.remove(LOCAL_CREDENTIALS_PATH)
    print(f"Arquivo de credenciais temporário '{LOCAL_CREDENTIALS_PATH}' removido.")

# Parar a sessão Spark
# spark.stop()